In [6]:

#------------------------------------------------------------------------------
# See: ~/.ipython/profile_default/startup/ipython_config.py
#------------------------------------------------------------------------------

import python_utils.python_utils as pyutils
import keras_utils as kutils
from helpers import *               # used to check train/test data
from pax_utils import s1s2_utils    # used to get s2 integral columns
from xy_s2integrals_models import * # ML models


Ready


In [7]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

nEventsTrain  = 100000
file_all      = '/project/lgrandi/dbarge/simulation/wimp/pax_v6.8.3/merged/merged_all_200000.pkl'
model_name_h5 = 'models/model_xy_s2integrals_dnn_elu_acc9915_epochs50_layers-None-None-None-None.h5'


In [9]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

file_out = './predictions/' + os.path.basename(model_name_h5).replace('.h5', '.pkl')

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[nEventsTrain:][:]

df_train_truth = df_all[:][['x', 'y']]
df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]

checkTrainingData(df_train_input, df_train_truth)



Input shape: (27826, 127)
Truth shape: (27826, 2)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
100000,0.000000,0.0,0.0,1.251830,0.000000,0.000000,0.000000,2.080982,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.376379,0.000000,0.222941
100001,0.889699,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.390214,0.0,0.000000,...,5.340566,0.0,69.842325,16.966499,2.908472,0.657984,0.00000,2.437548,16.276258,4.128640
100002,0.920286,0.0,0.0,2.590273,0.000000,1.302251,0.940790,0.000000,0.0,0.000000,...,7.989376,0.0,13.744347,25.277932,140.104943,62.470230,9.46162,9.785033,12.974688,41.039399
100003,0.000000,0.0,0.0,0.000000,6.720113,2.912992,2.984111,0.000000,0.0,0.000000,...,1.032516,0.0,0.000000,0.672962,1.554525,2.951680,0.00000,0.000000,0.000000,0.000000
100004,0.000000,0.0,0.0,5.068449,1.163184,4.138246,2.448707,0.000000,0.0,1.244048,...,0.000000,0.0,0.000000,1.110178,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000



Truth DataFrame:


,x,y
100000,7.332030,-30.591311
100001,-13.701462,12.687316
100002,7.042967,4.772081
100003,9.732044,31.983084
100004,17.004165,32.997236


In [10]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input.values)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")



Loaded Model: models/model_xy_s2integrals_dnn_elu_acc9915_epochs50_layers-None-None-None-None.h5
Predicted



In [11]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred


/project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [12]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

cols_out         = ['event_number', 'x', 'y', 'intr_x', 'intr_y', 'intr_x_nn', 'intr_y_nn']

df_out           = df_all[:][cols_out]
df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred

df_out.to_pickle(file_out)
df_test = pd.read_pickle(file_out)
display(df_test[0:5][:])
print()



,event_number,x,y,intr_x,intr_y,intr_x_nn,intr_y_nn,x_pred,y_pred
100000,156467,7.332030,-30.591311,5.640515,-25.802357,5.963322,-25.627516,7.946671,-31.179728
100001,156468,-13.701462,12.687316,-11.881086,10.920998,-12.208843,10.564307,-12.580153,11.383890
100002,156469,7.042967,4.772081,6.120559,4.440406,5.657052,3.909451,6.569002,4.653996
100003,156470,9.732044,31.983084,9.720888,28.202577,10.374315,27.733961,11.035250,30.875826
100004,156474,17.004165,32.997236,14.761349,28.682621,15.570843,28.051881,16.415651,30.378582
